Processing steps to create trial-unique beta maps from the CIMAQ fMRI data 
(memory task: image encoding), within a single subject:

Outputs (beta maps) are meant to be fed as features to a within-subject nilearn classifier.

Input:
    - task (event) files 
    - confound (motion, etc) files generated load_confound
    - preprocessed FMRIPrep data (4D .nii file)
        - note: data are not smoothed nor denoised)
Output: 1 map (3D .nii file) of beta (regression) weights for each trial, and 1 
concatenated 4D file of these 3D maps (trials ordered chronologically).

Note: comparing 2 different ways to model trials of no interest

version 1: Separate model for each trial
    - Trial of interest modelled as a separate condition (1 regressor)
    - All other trials modelled in either the Encoding or Control condition (2 regressors)
**Update: betas from version 1 lead to much better enc/ctl trial classification**

version 2: a separate model is built for each trial, with the trial of interest modelled as a separate condition (1 regressor), and all the other trials modelled as a single "other" condition (1 regressor)
**Update: betas from version2 lead to poorer enc/ctl trial classification**

Reference: how to derive beta maps for MVPA classification (Mumford et al., 2012):
https://www.sciencedirect.com/science/article/pii/S1053811911010081

Also creating contrasts per condition (to derive features for between-subject classification): 
 - Modeling enconding and control conditions across trials
     - 3 beta maps:
         - encoding (enc) , control (ctl), and encoding minus control (enc_minus_ctl)
 - Modeling control condition, as well as the encoding condition according to task performance:
    - miss and hit (post-scan image recognition performance)
    - 5 beta maps:
        - miss, hit hit_minus_miss, hit_minus_ctl, miss_minus_ctl
    - Modeling control condition & encoding condition according to task performance:
        - miss, wrong source, and correct source
    - 7 beta maps:
         - wrong_source, corr_source, cs_minus_ws, cs_minus_miss, ws_minus_miss, cs_minus_ctl, ws_minus_ctl


In [1]:
%matplotlib inline

In [2]:
import glob
import more_itertools
import nibabel
import nilearn
import numpy as np
import os
import pandas as pd
import re
import scipy
import sys


from load_confounds import Minimal
from joblib import parallel_backend
from numpy import nan as NaN
from matplotlib import pyplot as plt
from nibabel.nifti1 import Nifti1Image
from pathlib import Path
# from nilearn.glm.first_level import FirstLevelModel, check_events
# from nilearn.glm.first_level import make_first_level_design_matrix
# from nilearn import plotting as niplot
# from nilearn import image as nimage
# from nilearn.masking import apply_mask
# from nilearn.signal import clean
from tqdm import tqdm
from typing import Union

#libraries need to be installed in conda environment with pip install

In [4]:
from nibabel.nifti1 import Nifti1Image
from typing import Union

def chunks(lst, n):
    """ Yield successive n-sized chunks from lst. """
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def img_to_mask3d(source_img:Nifti1Image
                 ) -> Nifti1Image:
    import numpy as np
    from nilearn.image import new_img_like
    return new_img_like(ref_niimg=source_img,
                        data=np.where(source_img.get_fdata() \
                                      > 0, 1, 0))

def format_difumo(indexes:list,
                  target_img:Nifti1Image,
                  dimension:int=256,
                  resolution_mm:int=3,
                  data_dir:Union[str,os.PathLike]=None,
                  **kwargs
                  ) -> Nifti1Image:
    import numpy as np
    from nilearn.datasets import fetch_atlas_difumo
    from nilearn.masking import intersect_masks
    from nilearn import image as nimage
    difumo = fetch_atlas_difumo(dimension=dimension,
                                resolution_mm=resolution_mm,
                                data_dir=data_dir)
    rois = nimage.index_img(nimage.load_img(difumo.maps), indexes)
    rois_resampled = nimage.resample_to_img(source_img=nimage.iter_img(rois),
                                            target_img=target_img,
                                            interpolation='nearest',
                                            copy=True, order='F',
                                            clip=True, fill_value=0,
                                            force_resample=True)
    return intersect_masks(mask_imgs=list(map(img_to_mask3d,
                                              list(nimage.iter_img(rois_resampled)))),
                           threshold=0.0, connected=True)


#     return next((False if not os.path.exist(evpath)
#                  else evpath))


# def make_fit_1stlevel_glm(fmri_img:Union[str,os.PathLike,Nifti1Image],
#                           events:pd.DataFrame,
#                           confounds:pd.DataFrame=None,
#                           **kwargs):
#     short_events = events[['onset','trial_type','duration']].copy(deep=True)
# #     Add modulation column to events since ctl classification is 2x poorer than enc?
# #     short_events['modulation'] = [0.2 if 'Enc' in ttype else 0.9
# #                                   for ttype in short_events.trial_type]


#     glm_params = dict(t_r=fmri_img.header.get_zooms()[-1],
#                       mask_img=mask_img,
#                       drift_model=design_params['drift_model'],
#                       standardize=False,
#                       smoothing_fwhm=None,
#                       signal_scaling=False,
#                       noise_model='ar1',
#                       hrf_model=design_params['hrf_model'],
#                       minimize_memory=False)
#     design = make_first_level_design_matrix(**design_params)
#     model = FirstLevelModel(**glm_params).fit(fmri_img, design_matrices=design)
#     return model

#############################
# def trial_fmri(fmri_path:Union[str,os.PathLike, Nifti1Image],
#                events_path:Union[str,os.PathLike, pd.DataFrame],
#                sep:str='\t',
#                **kwargs):
#     from itertools import starmap
#     from nilearn import image as nimage
#     import pandas as pd
#     # Make pandas Intervals (b:list of beginnigs, e:list of ends)
#     mkintrvls = lambda b, e: list(starmap(pd.Interval,tuple(zip(b, e))))
#     fmri_img = nimage.load_img(fmri_path)
#     events = [events if isinstance(events, pd.DataFrame)
#                else pd.read_csv(events_path, sep=sep)][0]
#     t_r = fmri_img.header.get_zooms()[-1]
#     frame_times = np.arange(fmri_img.shape[-1]) * t_r
#     frame_intervals = mkintrvls(pd.Series(frame_times).values,
#                                 pd.Series(frame_times).add(t_r).values)
#     trial_ends=(events.onset+abs(events.onset -
#                                  events.offset)+events.isi).values
#     trial_intervals = mkintrvls(events.onset.values, trial_ends)
# #     trial_intervals = list(starmap(pd.Interval,tuple(zip(events.onset.values, trial_ends))))
#     bold_by_trial_indx = [[frame[0] for frame in enumerate(frame_intervals)
#                            if frame[1].left in trial] for trial in trial_intervals]
#     bold_by_trial = list(nimage.index_img(fmri_img, idx)
#                          for idx in bold_by_trial_indx)
#     event_list = events.loc[[item[0] for item in
#                               enumerate(bold_by_trial) if item != []]]
#     mem_labels, recall_labels = events_list.contidion, events_list.recognition_performance
#     return bold_by_trial, mem_labels, recall_labels
###############################

def trial_fmri(fmri_path:Union[str,os.PathLike, Nifti1Image],
               events_path:Union[str,os.PathLike, pd.DataFrame],
               sep:str='\t', t_r:float=None,
               **kwargs):
    from itertools import starmap
    from more_itertools import flatten
    from nilearn import image as nimage
    import pandas as pd
    # Make pandas Intervals (b:list of beginnigs, e:list of ends)
    mkintrvls = lambda b, e: list(starmap(pd.Interval,tuple(zip(b, e))))
    fmri_img = nimage.load_img(fmri_path)
    if not isinstance(events_path, pd.DataFrame):
        events = pd.read_csv(events_path, sep=sep)
    else:
        events = events_path
    t_r = [t_r if t_r is not None else
           fmri_img.header.get_zooms()[-1]][0]
    frame_times = np.arange(fmri_img.shape[-1]) * t_r
    frame_ends = pd.Series(frame_times).add(t_r).values
    frame_intervals = mkintrvls(pd.Series(frame_times).values,
                                frame_ends)
    trial_ends=(events.onset+abs(events.onset -
                                 events.offset)+events.isi).values
    trial_intervals = mkintrvls(events.onset.values, trial_ends)
    valid_trial_idx = [trial[0] for trial in enumerate(trial_intervals)
                       if trial[1].left<frame_intervals[-1].left]
    valid_trials = pd.Series(trial_intervals).loc[valid_trial_idx].values
#     trial_intervals = list(starmap(pd.Interval,tuple(zip(events.onset.values, trial_ends))))
    bold_by_trial_indx = [[frame[0] for frame in enumerate(frame_intervals)
                           if frame[1].left in trial] for trial in valid_trials]
    bold_by_trial = list(nimage.index_img(fmri_img, idx)
                         for idx in bold_by_trial_indx)
    valid_frame_intervals = [pd.Series(frame_intervals).loc[bold_idx].values
                             for bold_idx in bold_by_trial_indx]
    perfo_labels = events.iloc[valid_trial_idx].recognition_performance.fillna('Ctl')
    condition_labels = events.iloc[valid_trial_idx].trial_type
    stim_labels = events.iloc[valid_trial_idx].stim_file.fillna('Ctl').values
    categ_labels = events.iloc[valid_trial_idx].stim_category.fillna('Ctl').values
    return pd.DataFrame(tuple(zip(valid_trial_idx, bold_by_trial,
                                  bold_by_trial_indx, valid_trials,
                                  valid_frame_intervals, condition_labels,
                                  perfo_labels, stim_labels, categ_labels)),
                        columns=['trials', 'trial_niftis', 'fmri_frames',
                                 'trial_intervals', 'fmri_frame_intervals',
                                 'condition_labels', 'performance_labels',
                                 'stimuli_files', 'category_labels'])
#     return pd.DataFrame(tuple(zip(bold_by_trial, condition_labels, perfo_labels)))

In [18]:
def fetch_fmriprep_session(fmri_path:Union[str,os.PathLike],
                           events_dir:Union[str,os.PathLike],
                           strategy:str='Minimal',
                           task:str='memory',
                           space:str='MNI152NLin2009cAsym',
                           anat_mod:str='T1w',
                           lc_kws:dict=None,
                           apply_kws:dict=None,
                           clean_kws:dict=None,
                           design_kws:dict=None,
                           glm_kws:dict=None,
                           masker_kws:dict=None,
                           **kwargs):
    import load_confounds
    from inspect import getmembers
    from nilearn import image as nimage
    from sklearn.utils import Bunch
    from pathlib import Path
    
    from cimaq_decoding_params import _params
    from cimaq_decoding_utils import get_fmriprep_anat, get_fmriprep_mask
    from cimaq_decoding_utils import clean_fmri, get_events
    from cimaq_decoding_utils import get_tr, get_frame_times

    events_path = get_events(fmri_path, events_dir)
    if events_path is False:
        return False
    events = pd.read_csv(events_path, sep='\t').iloc[1:,:]    
    sub_id, ses_id = Path(fmri_path).parts[-4:-2]
    mask_path = get_fmriprep_mask(fmri_path)
    anat_path = get_fmriprep_anat(fmri_path)
    loader = dict(getmembers(load_confounds))[f'{strategy}']
    loader = [loader(**lc_kws) if lc_kws is not None
              else loader()][0]
    conf = loader.load(fmri_path)
    fmri_img, mask_img, anat_img = tuple(map(nimage.load_img,
                                             [fmri_path, mask_path,
                                              anat_path]))
    t_r, frame_times = get_tr(fmri_img), get_frame_times(fmri_img)
    if apply_kws is not None:
        _params.apply_defs.update(apply_kws)    
    if clean_kws is not None:
        _params.clean_defs.update(clean_kws)
    cleaned_fmri = clean_fmri(fmri_img, mask_img, confounds=conf,
                              apply_kws=_params.apply_defs,
                              clean_kws=_params.clean_defs)
#                               **_params.apply_defs)

    # Argument definitions for each preprocessing step
    target_shape, target_affine = mask_img.shape, cleaned_fmri.affine

    _params.glm_defs.update(dict(t_r=t_r, mask_img=mask_img,
                                 target_shape=target_shape,
                                 target_affine=target_affine))

    if design_kws is not None:
        _params.design_defs.update(design_kws)
    if masker_kws is not None:
        _params.masker_defs.update(masker_kws)
    if glm_kws is not None:
        _params.glm_defs.update(glm_kws)

    return Bunch(sub_id=sub_id, ses_id=ses_id, task=task, space=space,
                 fmri_path=fmri_path, mask_path=mask_path, events_path=events_path,
                 confounds_loader=loader, confounds=conf, confounds_strategy=strategy,
                 smoothing_fwhm=_params.apply_defs.smoothing_fwhm,
                 fmri_img=fmri_img, mask_img=mask_img,
                 cleaned_fmri=cleaned_fmri, events=events,
                 **Bunch(clean_defs=_params.clean_defs,
                         design_defs=_params.design_defs,
                         glm_defs=_params.glm_defs,
                         masker_defs=_params.masker_defs))


In [19]:
fmriprep_dir = '/data/simexp/cimaq_preproc/fmriprep/'
events_dir = '/data/simexp/CIMAQ_AS_BIDS/'
path00='/data/simexp/cimaq_preproc/fmriprep/sub-3002498/ses-V10/func/sub-3002498_ses-V10_task-memory_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'

sub00 = fetch_fmriprep_session(path00,events_dir)

In [31]:
sub00.vectors = trial_fmri(sub00.cleaned_fmri, sub00.events)


In [87]:
from nilearn import datasets
from sklearn.utils import Bunch
atlases_dir = '../../nilearn_atlases/difumo_atlases/'#/difumo_atlases/256/3mm/maps.nii.gz'

difumo_256 = get_difumo('../../nilearn_atlases/difumo_atlases/', 256, 3)



In [97]:
# import warnings
# warnings.simplefilter('ignore', category='VisibleDeprecationWarning')

from nilearn.input_data import NiftiMasker, NiftiMapsMasker, NiftiSpheresMasker


sub00.spheresmasker = NiftiSpheresMasker(tuple(map(tuple,difumo256.labels[['gm','wm','csf']])),
                                        radius=None, allow_overlap=True,
                                        **sub00.masker_defs)

sub00.mapsmasker = NiftiMapsMasker(difumo256.maps, mask_img=sub00.mask_img,
                                   resampling_target='mask',
                                   allow_overlap=True, **sub00.masker_defs)

sub00.regmasker = NiftiMasker(target_affine=sub00.cleaned_fmri.affine,
                              mask_strategy='whole-brain-template',
                              **sub00.masker_defs)
# help(NiftiMasker)
# (seeds, radius=None)


In [100]:
help(sub00.spheresmasker.transform_single_imgs)

Help on method transform_single_imgs in module nilearn.input_data.nifti_spheres_masker:

transform_single_imgs(imgs, confounds=None, sample_mask=None) method of nilearn.input_data.nifti_spheres_masker.NiftiSpheresMasker instance
    Extract signals from a single 4D niimg.
    
    Parameters
    ----------
    imgs : 3D/4D Niimg-like object
        See http://nilearn.github.io/manipulating_images/input_output.html
        Images to process. It must boil down to a 4D image with scans
        number as last dimension.
    
    confounds : CSV file or array-like or pandas DataFrame, optional
        This parameter is passed to signal.clean. Please see the related
        documentation for details.
        shape: (number of scans, number of confounds)
    
    sample_mask : Any type compatible with numpy-array indexing, optional
        Masks the niimgs along time/fourth dimension to perform scrubbing
        (remove volumes with high motion) and/or non-steady-state volumes.
        This p

In [110]:
from nilearn import image as nimage
sub00.mapsmasker.fit(sub00.cleaned_fmri)

NiftiSpheresMasker(allow_overlap=True,
                   seeds=((0.586695983486764, 0.368756961303868,
                           0.043084972984381),
                          (0.669211638091079, 0.254778712999749,
                           0.06971050882056),
                          (0.362251970608158, 0.062204248082339,
                           0.148603287738793),
                          (0.629230791189012, 0.290432238090537,
                           0.070572678551846),
                          (0.227904058238554, 0.688151064336859,
                           0.083951217965771),
                          (0.560710360527329, 0.36266...
                          (0.582794072304041, 0.350574534648094,
                           0.058785683416785),
                          (0.610322177238097, 0.310950019622802,
                           0.074369088438938),
                          (0.567642388970224, 0.244854281471568,
                           0.156329745067564),
         

In [169]:
testmean = nilearn.image.concat_imgs([nilearn.image.mean_img(trial)
                           for trial in sub00.vectors.trial_niftis])

In [171]:
testmean_signals = sub00.spheresmasker.transform_single_imgs(testmean)

In [112]:
spheres_trials = list(sub00.spheresmasker.transform_single_imgs(trial)
                      for trial in tqdm(sub00.vectors.trial_niftis))

100%|█████████████████████████████████████████| 116/116 [27:36<00:00, 14.28s/it]


In [113]:
sheres_trial_means = list(sub00.spheresmasker.fit_transform(nimage.mean_img(trial))
                          for trial in tqdm(sub00.vectors.trial_niftis))

100%|█████████████████████████████████████████| 116/116 [28:16<00:00, 14.63s/it]


In [ ]:
maps_trial_means = list(sub00.mapsmasker.fit_transform(nimage.mean_img(trial))
                          for trial in tqdm(sub00.vectors.trial_niftis))

In [ ]:
reg_trial_means = list(sub00.regmasker.fit_transform(nimage.mean_img(trial))
                       for trial in tqdm(sub00.vectors.trial_niftis))

In [481]:
sub00.events#.category_labels.replace({'K':'kitchen'}).str.lower().tolist()


,trial_number,trial_type,stim_id,position_correct,response_time,onset,offset,isi,duration,stim_file,stim_category,recognition_response,recognition_responsetime,position_response,position_responsetime,recognition_accuracy,position_accuracy,recognition_performance
1,5,Enc,Old56,6,1.6,20.1,23.1,0.5,3.0,sporting_boxing_gloves.bmp,sporting,1.0,7.0,4.0,2.4,1.0,0.0,Miss
2,6,Enc,Old10,5,1.3,23.6,26.6,1.0,3.0,animal_penguin.bmp,animal,1.0,2.3,3.0,1.7,1.0,0.0,Miss
3,7,Enc,Old24,6,1.2,27.6,30.6,1.0,3.0,food_softcheese.bmp,food,1.0,4.1,4.0,0.8,1.0,0.0,Miss
4,8,Enc,Old77,9,1.0,31.6,34.6,1.5,3.0,vegie_radish.bmp,vegie,2.0,NaN,NaN,NaN,0.0,NaN,FA
5,9,Enc,Old55,8,0.8,36.0,39.1,2.0,3.0,sporting_bicycle_old.bmp,sporting,1.0,1.7,1.0,0.9,1.0,0.0,Miss
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,116,Enc,Old48,6,0.7,694.4,697.4,6.0,3.0,musical_piano_old.bmp,musical,1.0,2.2,4.0,0.9,1.0,0.0,Miss
113,117,Ctl,NaN,9,0.7,703.4,706.5,10.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,118,Enc,Old18,5,1.0,717.0,720.0,5.0,3.0,food_cracker02.bmp,food,1.0,2.0,3.0,1.7,1.0,0.0,Miss
115,119,Enc,Old30,5,2.0,725.0,728.0,1.0,3.0,kitchen_cookie cutter.bmp,kitchen,1.0,2.0,3.0,1.7,1.0,0.0,Miss


In [437]:
# counts=.condition_labels.value_counts()
# (counts.min()/counts.max())/
# {'Enc': round(counts.max()/counts.sum(), 2),
#  'Ctl': round(counts.min()/counts.sum(), 2)}
# # from sklearn.svm import SVC, LinearSVC
help(LinearSVC)
# from sklearn.preprocessing import MinMaxScaler
# help(MinMaxScaler)
# sub00.vectors

Help on class LinearSVC in module sklearn.svm._classes:

class LinearSVC(sklearn.linear_model._base.LinearClassifierMixin, sklearn.linear_model._base.SparseCoefMixin, sklearn.base.BaseEstimator)
 |  LinearSVC(penalty='l2', loss='squared_hinge', *, dual=True, tol=0.0001, C=1.0, multi_class='ovr', fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0, random_state=None, max_iter=1000)
 |  
 |  Linear Support Vector Classification.
 |  
 |  Similar to SVC with parameter kernel='linear', but implemented in terms of
 |  liblinear rather than libsvm, so it has more flexibility in the choice of
 |  penalties and loss functions and should scale better to large numbers of
 |  samples.
 |  
 |  This class supports both dense and sparse input and the multiclass support
 |  is handled according to a one-vs-the-rest scheme.
 |  
 |  Read more in the :ref:`User Guide <svm_classification>`.
 |  
 |  Parameters
 |  ----------
 |  penalty : {'l1', 'l2'}, default='l2'
 |      Specifies t

In [477]:
# labels_list=events00.recognition_performance.fillna('Ctl').values
def classify_signals(signals, labels,
                     test_size:float=0.4,
                     n_tests:int=15,
                     **kwargs):
    import scipy
    import sklearn
    from sklearn.model_selection import train_test_split
    from sklearn.svm import SVC, LinearSVC
#     from sklearn.metrics import accuracy_score, classification_report
#     from sklearn.metrics import confusion_matrix, precision_score, f1_score
#     from sklearn.model_selection import cross_val_predict, cross_val_score
    from sklearn.preprocessing import MinMaxScaler
    scores = []
#         with parallel_backend('threading',n_jobs=-1):
    for test in range(n_tests):
        sub_svc = sklearn.svm.LinearSVC(max_iter=10000000,
                                        penalty='l2',
                                        loss='hinge',
#                                         multi_class='crammer_singer',
#                                         class_weight={'Enc': 1/round(counts.max()/counts.sum(),2),
#                                                       'Ctl': 1/round(counts.min()/counts.sum(),2)},
#                                         intercept_scaling=0.1,
                                        class_weight='balanced',
                                        fit_intercept=True)
#         scaler = MinMaxScaler()
#         scaler.fit(signals)
#         signals = scaler.transform(signals)
        X_train, X_test, y_train, y_test = train_test_split(
            signals, labels, # x & y
            test_size = test_size, 
            shuffle = True,
            stratify = labels)
        sub_svc.fit(X_train, y_train)
        y_pred = sub_svc.predict(X_test) # classify age class using testing data
        print(len(list(filter(None, y_pred==y_test)))/len(y_test))
        print(y_pred)
        scores.append(sub_svc.score(X_test, y_test))
    return np.mean(scores).round(2)

In [478]:
stim_categs = sub00.vectors.category_labels.replace({'K':'kitchen'}).str.lower()
classify_signals(signals=testmean_signals,
                labels=sub00.vectors.performance_labels.tolist())

0.5531914893617021
['Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Ctl' 'Miss' 'Miss' 'Ctl' 'Miss' 'Ctl'
 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Ctl' 'Miss'
 'Miss' 'Miss' 'Miss' 'Ctl' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss'
 'Ctl' 'Miss' 'Miss' 'Ctl' 'Ctl' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss'
 'Miss' 'Miss' 'Miss' 'Miss' 'Miss']
0.574468085106383
['FA' 'FA' 'Miss' 'Ctl' 'Miss' 'Miss' 'Miss' 'FA' 'Miss' 'Miss' 'Miss'
 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'FA'
 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss'
 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss'
 'FA' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss']
0.5531914893617021
['Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss'
 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'Miss'
 'Miss' 'Miss' 'Miss' 'FA' 'Miss' 'FA' 'Miss' 'Miss' 'Miss' 'Miss' 'FA'
 'Miss' 'Miss' 'Miss' 'Miss' 'Miss' 'FA' 'Miss' 'Miss' 'FA' 'Miss' 'FA'
 'Miss'

0.55

In [ ]:
# cimaq_files = fetch_cimaq(fmriprep_dir = '/data/simexp/cimaq_preproc/fmriprep/',
#                           events_dir = '/data/simexp/CIMAQ_AS_BIDS/',
#                           atlases_dir = '../../nilearn_atlases/')

In [ ]:
from nilearn import image as nimage
sub00.maps_masker = NiftiMapsMasker(difumo256.maps, **sub00.masker_params)
sub00.maps_labels = difumo256.labels.difumo_names
sub00.vectors.time_series = [sub00.maps_masker.fit_transform(nimage.mean_img(row[1].trial_niftis))
                             for row in sub00.vectors.iterrows()]

In [ ]:
from nilearn.connectome import ConnectivityMeasure as CM
for time_series in sub00.stackdx_ts:
    conn_meas = CM(kind='correlation').fit_transform([time_series.reshape(-1,1)])
    print(conn_meas)
# sub00.vectors.corr_mats = [np.fill_diagonal(CM(kind='correlation').fit_transform([time_series.reshape(-1,1)])[0],0)
# #                            for time_series in sub00.stackdx_ts]

In [ ]:

# difumo_masker = NiftiMapsMasker(maps_img=difumo256.maps, standardize=False,
#                                 memory='nilearn_cache', verbose=0)

def get_corr_mat(fmri_img, masker, cond_idx:list=None,
                 **kwargs):
    import numpy as np
    from nilearn import plotting as niplot
    from nilearn.connectome import ConnectivityMeasure as CM
    from nilearn.image import index_img
    from nilearn.input_data import NiftiMapsMasker
    if cond_idx is not None:
        fmri_img = index_img(fmri_img, cond_idx)
    time_series = masker.fit_transform(fmri_img)
    correlation_matrix = CM(kind='correlation').fit_transform([time_series])[0]
    np.fill_diagonal(correlation_matrix, 0) # Mask out the major diagonal
    return time_series, correlation_matrix

# [sub00.__setattr__(*attrs) for attrs in tuple(zip(['time_series', 'correlation_matrix'],
#                                           get_corr_mat(sub00.cleaned_fmri,
#                                                        sub00.maps_masker,
#                                                        cond_idx)))]
sub00.enc_ctl_timeseries, sub00.enc_ctl_corr_mat = \
    get_corr_mat(sub00.cleaned_fmri, sub00.maps_masker, )
#     niplot.plot_matrix(correlation_matrix, labels=difumo256.labels,
#                          colorbar=True, vmax=0.8, vmin=-0.8)

In [ ]:
sub00.[vectors]

In [ ]:
sub00.vectors[['trial_arrays', 'performance_labels']]

In [ ]:

sub00.enc_ts = list(item[0] for item in tuple(zip(sub00.vectors.time_series,
                                   sub00.vectors.condition_labels))
              if item[1]=='Enc')

In [ ]:
sub00.ctl_ts = list(item[0] for item in tuple(zip(sub00.vectors.time_series,
                                   sub00.vectors.condition_labels))
              if item[1]=='Ctl')

In [ ]:
sub00.vectors['time_series'] = [sub00.vectors.time_series[:,:,list(map(int,row[1].fmri_frames))]
                        for row in sub00.vectors.iterrows()]

In [ ]:
# list(np.nditer(sub00.vectors.time_series, itershape=(0,0,1))).__len__()
# help(np.nditer)
# sub00.vectors.time_series=np.array(sub00.vectors['time_series'].values.tolist())
# sub00.vectors.time_series[:,:,0].shape
sub00.vectors['trial_arrays'] = [sub00.vectors.time_series[:,:,ind]
                                  for ind in sub00.vectors.index]
cov_estimator=sklearn.svm.LinearSVC(max_iter=1000000,
                                                                penalty='l2', loss='hinge',
                                                                class_weight='balanced'),

In [ ]:
def get_corr_mat_from_ts(time_series, labels):
    import numpy as np
    from nilearn.connectome import ConnectivityMeasure as CM
    correlation_matrix = CM(kind='correlation').fit_transform([time_series],
                                                              labels)
    np.fill_diagonal(correlation_matrix, 0) # Mask out the major diagonal
    return correlation_matrix


In [ ]:
labels=np.array([difumo256_labels]*goodshape.shape[1]).T
signals=goodshape
labels.shape,signals.shape

In [ ]:
get_corr_mat_from_ts(time_series=signals, labels=labels)

In [ ]:
np.array([difumo256_labels]*goodshape.shape[1]).T.shape, goodshape.shape

In [ ]:
from nilearn.connectome import ConnectivityMeasure as CM
# help(CM)
goodshape=sub00.vectors.time_series.transpose(1,-1,0)[0]
#.shape#[-1,::].shape for ind in sub00.vectors.index].__len__()

In [ ]:
# get_corr_mat_from_ts(time_series=sub00.vectors.time_series)
# cond_ts['Enc_ts'].shape
difumo256_labels = pd.DataFrame(difumo256.labels).difumo_names.values
[get_corr_mat_from_ts(sub00.vectors.time_series[:,ind,:], difumo256_labels) for ind in sub00.vectors.index]

In [ ]:
perfo_ts = dict(tuple((f'{outcome}_ts', sub00.vectors.loc[sub00.vectors['performance_labels'] == outcome].trial_arrays.values)
                      for outcome in sub00.vectors.performance_labels.unique()))

In [ ]:
cond_ts = dict(tuple((f'{cond}_ts', sub00.vectors.loc[sub00.vectors['condition_labels'] == cond].trial_arrays.values)
                      for cond in sub00.vectors.condition_labels.unique()))

In [ ]:
cond_ts['Enc_ts'][0]

In [ ]:
[type(sub00[key]) for key in ['enc_ts', 'ctl_ts', 'Miss_ts', 'Hit_ts', 'Ctl_ts', 'FA_ts']]

In [ ]:
sub00.vectors['time_series'].values.flatten().shape

In [ ]:
sub00.enc_ctl_timeseries, sub00.enc_ctl_corr_mat = \
    get_corr_mat(sub00.cleaned_fmri, sub00.maps_masker, )

In [ ]:
# list(key for key in sub00.keys() if key.endswith('_ts'))
cmdict = dict(tuple((tskey.split('_')[0]+'_corr_mat',get_corr_mat_from_ts(sub00[tskey]))
                    for tskey in ['Miss_ts',
                                  'Hit_ts', 'Ctl_ts', 'FA_ts']))

# sub00.vectors.stimuli_files.value_counts().values

In [ ]:
[sub00.__setattr__(*attrs) for attrs in tuple(zip(['time_series', 'correlation_matrix'],
                                          get_corr_mat(sub00)))]

In [ ]:
big_bunch = sklearn.utils.Bunch(**dict(('-'.join([i.sub_id,i.ses_id]), i)
                                       for i in filter(None,cimaq_bunch)))

In [ ]:
# cleaned_bold55, mask55, anat_img55, events55 = 
test_subject = load_fmriprep_session(cimaq_files.bolds[55], anat_mod='T1w',
                                     events_dir='/data/simexp/CIMAQ_AS_BIDS/')


In [ ]:
from nilearn.input_data import NiftiMapsMasker
from nilearn import datasets
atlases_dir = '../../nilearn_atlases/'
difumo256 = datasets.fetch_atlas_difumo(256, 3, data_dir=atlases_dir)
difumo_masker55 = NiftiMapsMasker(maps_img=difumo256.maps, standardize=False,
                         memory='nilearn_cache', verbose=0)

time_series55 = difumo_masker55.fit_transform(cleaned_bold55)

In [ ]:
from nilearn.connectome import ConnectivityMeasure
correlation_measure = ConnectivityMeasure(kind='correlation')
correlation_matrix = correlation_measure.fit_transform([time_series55])[0]

# Display the correlation matrix
import numpy as np
from nilearn import plotting
# Mask out the major diagonal
np.fill_diagonal(correlation_matrix, 0)
plotting.plot_matrix(correlation_matrix, labels=difumo256.labels.difumo_names,
                     colorbar=True, vmax=0.8, vmin=-0.8)

In [ ]:
# correlation_matrix.__dir__()
mat_labels = pd.DataFrame(difumo256.labels).difumo_names.values
corrdf = pd.DataFrame(correlation_matrix, columns=mat_labels,index=mat_labels)
display(corrdf.max(), corrdf.min())

In [ ]:
idxs, bbt55 = trial_fmri(cleaned_bold55, events55)
mbbt55 = [nilearn.image.mean_img(im) for im in bbt55]
mean_signals55 = nilearn.masking.apply_mask(mbbt55, mask55)
fmri_bt = nilearn.image.concat_imgs(mbbt55)

In [ ]:
n_jobs = 1

# Define the cross-validation scheme used for validation.
# Here we use a KFold cross-validation on the session, which corresponds to
# splitting the samples in 4 folds and make 4 runs using each fold as a test
# set once and the others as learning sets
from sklearn.model_selection import KFold
cv = KFold(n_splits=4)

import nilearn.decoding
# The radius is the one of the Searchlight sphere that will scan the volume
searchlight = nilearn.decoding.SearchLight(
    mask55,
    process_mask_img=mask55,
    radius=9.0, n_jobs=n_jobs,
    verbose=1, cv=cv)
searchlight.fit(fmri_bt, idxs)

In [ ]:
from nilearn.input_data import NiftiMasker

# For decoding, standardizing is often very important
nifti_masker = NiftiMasker(mask_img=mask55, sessions=session,
                           standardize=True, memory='nilearn_cache',
                           memory_level=1)
fmri_masked = nifti_masker.fit_transform(fmri_img)

from sklearn.feature_selection import f_classif
f_values, p_values = f_classif(fmri_masked, y)
p_values = -np.log10(p_values)
p_values[p_values > 10] = 10
p_unmasked = get_data(nifti_masker.inverse_transform(p_values))

In [ ]:
from nilearn import plotting as niplot
# coords = difumo256.region_coords

# We threshold to keep only the 20% of edges with the highest value
# because the graph is very dense
niplot.plot_connectome(correlation_matrix,
                       edge_threshold="80%", colorbar=True)

plotting.show()

In [ ]:
idxs=idxs.fillna('Ctl').replace(dict(enumerate(idxs.unique())))

In [ ]:
dict(enumerate(idxs.unique()))

In [ ]:
# sorted(dir(torch))
torch.Tensor(mean_signals55)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# Equates to one random 28x28 image
random_data = torch.rand((1, 1, 28, 28))


In [ ]:
dir(torch.nn.functional)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# Equates to one random 28x28 image
# random_data = torch.rand((1, 1, 28, 28))

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # First 2D convolutional layer, taking in 1 input channel (image),
        # outputting 32 convolutional features, with a square kernel size of 3
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        # Second 2D convolutional layer, taking in the 32 input layers,
        # outputting 64 convolutional features, with a square kernel size of 3
        self.conv2 = nn.Conv2d(32, 64, 3, 1)

        # Designed to ensure that adjacent pixels are either all 0s or all active
        # with an input probability
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)

        # First fully connected layer
        self.fc1 = nn.Linear(9216, 128)
        # Second fully connected layer that outputs our 10 labels
        self.fc2 = nn.Linear(128, 10)

my_nn = Net()
print(my_nn)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    # x represents our data
    def forward(self, x):
        # Pass data through conv1
        x = F.max_pool2d(F.relu(self.conv2(F.relu(self.conv1(x)))), 2)
        # Use the rectified-linear activation function over x
        # Run max pooling over x
        # Pass data through dropout1
        x = F.relu(self.fc1(torch.flatten(self.dropout1(x), 1)))
        # Flatten x with start_dim=1
        # Pass data through fc1
        x = self.fc2(self.dropout2(x))
        # Apply softmax to x
        return F.log_softmax(x, dim=1)
    
# Equates to one random 28x28 image
# random_data = torch.rand((1, 1, 28, 28))

my_nn = Net()
result = my_nn(tuple(zip(torch.Tensor(mean_signals55), idxs.values.tolist())))
print(result)

In [ ]:
import torch
import torchaudio
import torchvision
data_loader = torch.utils.data.DataLoader(yesno_data,
                                          batch_size=1,
                                          shuffle=True)

In [ ]:
classify_signals(signals=time_series, labels=labels)

In [ ]:
import scipy
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC

signals=mean_signals55
labels=idxs
sub_svc = sklearn.svm.LinearSVC(max_iter=1000000,
                                penalty='l2', loss='hinge',
                                class_weight='balanced')
with parallel_backend('threading',n_jobs=-1):
    X_train, X_test, y_train, y_test = train_test_split(
        signals, labels, # x & y
        test_size = 0.5, 
        shuffle = True,
        stratify = labels)
    sub_svc.fit(X_train, y_train)
    print(sub_svc.score(X_test, y_test))

In [ ]:
y_pred = sub_svc.predict(X_test)

In [ ]:
classify_signals(signals=mean_signals55, labels=idxs)

In [ ]:
def linear_classif(X, Y,
                   svm_kws:dict=None,
                   train_kws:dict=None,
                   test_size:float=0.5,
                   **kwargs):
    import scipy
    import sklearn
    from sklearn.model_selection import train_test_split
    from sklearn.svm import SVC, LinearSVC
    from sklearn.metrics import accuracy_score, classification_report
    from sklearn.metrics import confusion_matrix, precision_score, f1_score
    from sklearn.model_selection import cross_val_predict, cross_val_score
    from sklearn.preprocessing import MinMaxScaler
    svm_defs = dict(max_iter=1000000, penalty='l2',
                    loss='hinge', class_weight='balanced')
    if svm_kws is not None:
        svm_defs.update(svm_kws)
    train_defs = dict(test_size=test_size,
                        shuffle=True, stratify=Y)
    if train_kws is not None:
        train_defs.update(train_kws)
    sub_svc = sklearn.svm.LinearSVC(**svm_defs)
    X_train, X_test, y_train, y_test = \
        train_test_split(X, Y,**train_defs)
    return sub_svc.fit(X_train, y_train).score(X_test, y_test)

In [ ]:
import SimpleITK as sitk
import numpy as np

# A path to a T1-weighted brain .nii image:
# t1_fn = './brain_t1_0001.nii'

# Read the .nii image containing the volume with SimpleITK:
# sitk_t1 = sitk.ReadImage(mbbt55)
tensorlist = [sitk.GetArrayFromImage(sitk.ReadImage(sub_img))
              for sub_img in mbbt55]
# and access the numpy array:
# t1 = sitk.GetArrayFromImage(sitk_t1)

In [ ]:
mkintrvls = lambda b, e: list(starmap(pd.Interval,tuple(zip(b, e))))
from itertools import starmap
t_r = cleaned_bold55.header.get_zooms()[-1]
frame_times = np.arange(cleaned_bold55.shape[-1]) * t_r
frame_ends = pd.Series(frame_times).add(t_r).values
frame_intervals = mkintrvls(pd.Series(frame_times).values,
                            frame_ends)
trial_ends = (events55.onset+abs(events55.onset -
                                 events55.offset)+events55.isi).values
trial_intervals = mkintrvls(events55.onset.values, trial_ends)
valid_frame_intervals = [intv for intv in frame_intervals if
                         intv.right<trial_intervals[-1].left]
valid_trial_intervals = [intv for intv in trial_intervals if
                         intv.right<frame_intervals[-1].left]
len(valid_trial_intervals)
# frame_intervals

In [ ]:
len(valid_frame_intervals)

In [ ]:
signals55 = nilearn.masking.apply_mask(mbbt55, mask55)

In [ ]:
signals55.shape

In [ ]:
from nilearn import image as nimage
nimage.load_img(nimage.load_img(cleaned_bold)).header.get_zooms()[-1]

In [ ]:
cimaq_files.events.apply(os.path.basename)

In [ ]:
mean_bold_by_trial = list(nimage.mean_img(bbt) for bbt in bold_by_trial)

In [ ]:
mean_signal_by_trial = apply_mask(mean_bold_by_trial, mask)

In [ ]:
classify_signals(mean_signal_by_trial, labels_list)

In [ ]:
trial_based_fmri = nimage.concat_imgs([nimage.mean_img(nimage.index_img(sub00.cleaned_fmri,frames))
                                       for frames in sub00.vectors.fmri_frames])

In [ ]:
help(regmasker.fit_transform)

In [ ]:
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.first_level import make_first_level_design_matrix
help(make_first_level_design_matrix)

In [ ]:

sub00.events.trial_number.astype(str).str.zfill(3)
padnums = lambda l: [str(x).zfill(3) for x in l]
list(map(padnums,sub00.events.groupby('trial_type')['trial_type'].groups.values()))

In [ ]:
(sub00.events.copy(deep=True).trial_number.astype(str).str.zfill(3)+sub00.events.trial_type).values

In [ ]:
# def custom_design():
sub00.condition_events = sub00.events.copy(deep=True)[['onset','trial_type','duration']]
sub00.performance_events = sub00.events.copy(deep=True).drop('trial_type',axis=1).rename(
                               {'recognition_performance':'trial_type'},
                                axis=1)[['onset','trial_type','duration']].fillna('Ctl')
sub00.global_events = sub00.events.copy(deep=True)
sub00.global_events['trial_type'] = (sub00.global_events.trial_number.astype(str).str.zfill(3) +
                                     sub00.global_events.trial_type).values
sub00.global_events = sub00.global_events[['onset','trial_type','duration']]
# sub00.global_events = sub00.events.copy(deep=True)[['onset','trial_type','duration']].fillna('Ctl')
sub00.global_events['trial_type'] = sub00.events.trial_number.astype(str)+sub00.events.trial_type
sub00.design_params = dict(frame_times=(np.arange(sub00.cleaned_fmri.shape[-1]) *
                                        sub00.cleaned_fmri.header.get_zooms()[-1]),
                           drift_model=None,
                           hrf_model='spm')
sub00.condition_design = make_first_level_design_matrix(events=sub00.condition_events,
                                                        **sub00.design_params)
sub00.performance_design = make_first_level_design_matrix(events=sub00.performance_events,
                                                          **sub00.design_params)
sub00.global_design = make_first_level_design_matrix(events=sub00.global_events,
                                                          **sub00.design_params)

In [ ]:
display(nilearn.plotting.plot_design_matrix(sub00.condition_design),
        nilearn.plotting.plot_design_matrix(sub00.performance_design),
        nilearn.plotting.plot_design_matrix(sub00.global_design))

In [ ]:
sub00['glm_params']=dict(t_r=sub00.cleaned_fmri.header.get_zooms()[-1],
                          mask_img=sub00.full_mask_img,
                          drift_model=sub00.design_params['drift_model'],
                          standardize=False,
                          smoothing_fwhm=None,
                          signal_scaling=(0,1), # Other choices: 0, 1, False
                          noise_model='ar1',
                          hrf_model=sub00.design_params['hrf_model'],
                          subject_label=sub00.sub_id,
                          target_affine=sub00.cleaned_fmri.affine,
                          target_shape=sub00.full_mask_img.shape,
                          n_jobs=-1,
                          minimize_memory=False)


In [ ]:
sub00.condition_glm

In [ ]:
def make_fit_1stlevel_glm(fmri_img:Union[str,os.PathLike,Nifti1Image],
                          events:pd.DataFrame,
                          confounds:pd.DataFrame=None,
                          **kwargs):
#     short_events = events[['onset','trial_type','duration']].copy(deep=True)
#     Add modulation column to events since ctl classification is 2x poorer than enc?
#     short_events['modulation'] = [0.2 if 'Enc' in ttype else 0.9
#                                   for ttype in short_events.trial_type]
#     sub00.condition_events = sub00.events[['onset','trial_type','duration']]
#     sub00.performance_events = sub00.events.rename({'recognition_performance':'trial_type'},
#                                              axis=1)[['onset','trial_type','duration']]
#     design_params = dict(frame_times=(np.arange(sub00.cleaned_fmri.shape[-1]) *
#                                       fmri_img.header.get_zooms()[-1]),
#                          events=sub00.condition_events[['onset','trial_type','duration']],
#                          drift_model=None,
#                          hrf_model='spm')

glm_params = dict(t_r=sub00.cleaned_fmri.header.get_zooms()[-1],
                  mask_img=sub00.full_mask_img,
                  drift_model=design_params['drift_model'],
                  standardize=False,
                  smoothing_fwhm=None,
                  signal_scaling=(0,1), # Other choices: 0, 1, False
                  noise_model='ar1',
                  hrf_model=design_params['hrf_model'],
                  subject_label=sub00.sub_id,
                  target_affine=sub00.cleaned_fmri.affine,
                  target_shape=sub00.full_mask_img.shape,
                  n_jobs=-1,
                  minimize_memory=False)
    # Signal extraction methods: 
    # compute_contrast(self, contrast_def, stat_type=None, output_type='z_score')
    # DONE fit(self, run_imgs, events=None, confounds=None, design_matrices=None, bins=100)
    # predicted(), r_square(), residuals
    # fit_transform(self, X, y=None, **fit_params)
#     design = make_first_level_design_matrix(**design_params)
    model = FirstLevelModel(**glm_params).fit(sub00.cleaned_fmri,
                                              design_matrices=design)
    
    return model

In [ ]:
sub00.condition_glm.design_matrices_[0].columns[:-1].tolist()

In [ ]:
sub00.condition_contrasts = sklearn.utils.Bunch(**dict(tuple((col,sub00.condition_glm.compute_contrast(contrast_def=col,
                                                                                 output_type='all'))
                      for col in sub00.condition_glm.design_matrices_[0].columns[:-1].tolist())))
sub00.performance_contrasts = sklearn.utils.Bunch(**dict(tuple((col,sub00.performance_glm.compute_contrast(contrast_def=col,
                                                                                   output_type='all'))
                      for col in sub00.performance_glm.design_matrices_[0].columns[:-1].tolist())))
sub00.global_contrasts = sklearn.utils.Bunch(**dict(tuple((col,sub00.global_glm.compute_contrast(contrast_def=col,
                                                                           output_type='all'))
                      for col in sub00.global_glm.design_matrices_[0].columns[:-1].tolist())))

In [ ]:
# import copy
# condcp = copy.deepcopy(sub00.condition_contrasts)
# def contrast2signal(contrast)
sub00.condition_glm.masker_

In [ ]:
# pd.DataFrame.from_dict(condcp)
globalcp = copy.deepcopy(sub00.global_contrasts)
globdf = pd.DataFrame.from_dict(globalcp)
# help(sklearn.utils.Bunch)#(**condcp).Enc.effect_size

In [ ]:
# # [setattr(sub00, f'{attr}_glm', FirstLevelModel(**sub00.glm_params))
# #                         for attr in ['condition','performance','global']]
# [sub00[f'{attr}_glm'].fit(sub00.cleaned_fmri,
#                         design_matrices=sub00[f'{attr}_design'])
#  for attr in ['condition','performance','global']]
fxsize_globaltest = 

In [ ]:
# # (trial_based_fmri)
mapsmasker=NiftiMapsMasker(maps_img=difumo256.maps,
                           mask_img=sub00.full_mask_img,
                           standardize_confounds=False,
                           resampling_target='data')
mapsmasker.fit(sub00.cleaned_fmri,sub00.vectors.performance_labels)

regmasker = NiftiMasker(mask_img=sub00.full_mask_img,
                        mask_strategy='whole-brain-template',
                        target_affine=sub00.cleaned_fmri.affine,
                        t_r=sub00.cleaned_fmri.header.get_zooms()[-1],
                        target_shape=sub00.full_mask_img.shape,
                        standardize_confounds=False)
# trial_based_signals = mapsmasker.transform_single_imgs(trial_based_fmri)
regmasker.fit(sub00.cleaned_fmri,
              sub00.events[['onset','trial_type','duration']])
# single_signals = regmasker.transform_single_imgs(sub00.cleaned_fmri)
# fit_trans_signals = regmasker.fit_transform(X=sub00.cleaned_fmri,
#                                             y=sub00.events[['onset','trial_type','duration']])

# 65*77*60==300300
# help(NiftiMapsMasker)

In [ ]:
sub00.condition_contrasts

In [ ]:
# globdf.loc[f'{stat}_sig']=

globsignaldf = sklearn.utils.Bunch(pd.DataFrame([[regmasker.fit_transform(signal) for signal
                             in globdf.loc[stat].values]
                             for stat in globdf.index.tolist()],
                            index=[ind+'_sig' for ind in globdf.index.tolist()],
                            columns=globdf.columns))
# globdf.loc['effect_size'].values

In [ ]:
last_attempt = regmasker.fit_transform(globalcp.loc['effect_size'])

In [ ]:
globalcp

In [ ]:
X[0].shape

In [ ]:
import torch
from googlenet_pytorch import GoogLeNet 
model = GoogLeNet.from_pretrained('googlenet')

# ... image preprocessing as in the classification example ...
inputs = torch.randn(1, 3, 224, 224)

print(inputs.shape) # torch.Size([1, 3, 224, 224])

# features = model.extract_features(inputs)
# torch.Size([1, 1024, 7, 7])
# print(features.shape)

In [ ]:
# [i for i in sub00.condition_contrasts]
enc_map=sub00.condition_contrasts['Enc']['effect_size']
ctl_map=sub00.condition_contrasts['Ctl']['effect_size']
ctl_map

In [ ]:
# tst=list(next(iter(item)) for item in )
# tst[0].shape
# globalcp.iloc[-2][0].shape
os.listdir('../../WMStim')

In [ ]:
os.listdir('../../../fnadeau/')

In [ ]:
# import shelve
# # help(shelve.Shelf)
# with open(,'w') as savedst:
#           pickle.dump(savedst,sub00)
import loadutils as lu
lu.save_pickle('../../../fnadeau/sub00_bunch.pickle',sub00)

In [ ]:
from nilearn.connectome import ConnectivityMeasure as CM
# X = list(next(iter(item)) for item in globalcp.iloc[-2])
X = last_attempt.T
# y = [col.lstrip(digits) for col in globalcp.columns]
#
X = X = trial_based_signals
# y = pd.DataFrame(difumo256.labels).difumo_names.values

linsvm = sklearn.svm.LinearSVC(max_iter=1000000,
                                    penalty='l2',
                                    loss='hinge',
                                    class_weight='balanced')

linsvm.fit(X,y)


In [ ]:
# linsvm.__dict__
from string import digits
y = [col.lstrip(digits) for col in globalcp.columns]
y

In [ ]:
correlation_measure = CM(cov_estimator=linsvm,
                         kind='correlation')
# correlation_measure.fit(X,y)
correlation_matrix2 = correlation_measure.fit_transform(X,y)

# # Display the correlation matrix
# import numpy as np
# from nilearn import plotting
# # Mask out the major diagonal
# np.fill_diagonal(correlation_matrix2, 0)
# correlation_matrix2.shape
# plotting.plot_matrix(correlation_matrix2, labels=y,
# #                      figure=(128,128),
#                      colorbar=True, vmax=0.8, vmin=-0.8)

In [ ]:
from nilearn.input_data import NiftiMasker, NiftiMapsMasker, NiftiSpheresMasker
help(NiftiMasker)

In [ ]:
corrdf = pd.DataFrame(correlation_matrix, index=difumo256.labels.difumo_names,
                              columns=difumo256.labels.difumo_names)
# corrdf.where(corrdf==corrdf.max()).dropna(how='all',axis=0)
# [(row[1].index,row[1].name) for row in corrdf.iterrows()]


In [ ]:
from nilearn.connectome import ConnectivityMeasure as CM
help(CM)

In [ ]:
# difumo256.labels.difumo_names.shape
trial_based_signals.T.shape,difumo256.labels['difumo_names'].shape

In [ ]:
# dir(linsvm)
linsvm.__dict__








In [ ]:
# 60%/40% split with LinearSVC, 50-50 with NuSVC
import scipy
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, precision_score, f1_score
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.preprocessing import MinMaxScaler

def classify_bmap(beta_map, labels,
                  mask=None, n_tests=15):
    from nilearn import image as nimage
    from nilearn.masking import apply_mask
# max_iter=1000000, dual=True, tol=1E-4
    sub_svc = sklearn.svm.LinearSVC(max_iter=1000000,
                                    penalty='l2',
                                    loss='hinge',
                                    class_weight='balanced',
#                                     fit_intercept=True,
#                                     intercept_scaling=1.0
#                                     probability=True,
                                ) # 0.73 avg acc up to now
    scores = []
    for test in range(n_tests):
        X_enc_ctl = beta_map
#         X_enc_ctl = apply_mask(list(nimage.iter_img(beta_map)), mask)
#         X_enc_ctl = [[np.float(s) for s in scipy.fft.fft(sig)]
#                      for sig in X_enc_ctl]
        y_enc_ctl = labels
        X_train, X_test, y_train, y_test = train_test_split(
            X_enc_ctl, # x
            y_enc_ctl, # y
            test_size = 0.4, 
            shuffle = True, # shuffle dataset before splitting
            stratify = y_enc_ctl, # keep distribution of conditions consistent betw. train & test sets
            #random_state = 123  # if set number, same shuffle each time, otherwise randomization algo
            )
        # Test model on unseen data from the test set
        sub_svc.fit(X_train, y_train)
        y_pred = sub_svc.predict(X_test) # classify age class using testing data
        print(len(list(filter(None, y_pred==y_test)))/len(y_test))
        print(y_pred)
        scores.append(sub_svc.score(X_test, y_test)) # get accuracy
    print(np.mean(scores).round(2))
#     cr = classification_report(y_pred=y_pred, y_true=y_test).precision # get prec., recall & f1
#     crdf = pd.DataFrame(tuple(filter(None,[line.split()[:-2]
#                                            for line in cr.splitlines()]))).T.set_index(0).T
#     precision_df.append(crdf)



In [ ]:
classify_bmap(trial_based_signals.T,labels=sub00.vectors.performance_labels)

In [ ]:
import tensorflow as tf

# tf.keras.applications.vgg16.VGG16(
#     include_top=True, weights='imagenet',
#     input_tensor=None,
#     input_shape=None, pooling=None, classes=1000,
#     classifier_activation='softmax')


In [ ]:
# Load all data into memory
data = load_data(all_filenames, tf.estimator.ModeKeys.TRAIN, reader_params)

# Create placeholder variables and define their shapes (here, 
# we input a volume image of size [128, 224, 244] and a single
# channel (i.e. greyscale):
x = tf.placeholder(reader_example_dtypes['features']['x'], 
                   [None, 128, 224, 224, 1])
y = tf.placeholder(reader_example_dtypes['labels']['y'], 
                   [None, 1])

# Create a tf.data.Dataset
dataset = tf.data.Dataset.from_tensor_slices((x, y))
dataset = dataset.repeat(None)
dataset = dataset.batch(batch_size)
dataset = dataset.prefetch(1)

# Create an iterator
iterator = dataset.make_initializable_iterator()
nx = iterator.get_next()

with tf.train.MonitoredTrainingSession() as sess_dict:
    
    sess_dict.run(iterator.initializer, 
               feed_dict={x: data['features'], y: data['labels']})
    
    for i in range(iterations):
        # Get next features/labels pair
        dict_batch_feat, dict_batch_lbl = sess_dict.run(nx)

In [ ]:
from keras.applications.vgg16 import preprocess_input, VGG16
model = VGG16(include_top=True, weights='imagenet',
              input_tensor=None,
              input_shape=None, pooling=None, classes=1000,
              classifier_activation='softmax')

def make_tf_inputs(inpt_list:list)->list:
    from keras.preprocessing.image import img_to_array, load_img
# load an image from file
    images = [img_to_array(load_img(fpath, target_size=(224, 224)))
              for fpath in inpt_list]
    images = [preprocess_input(image.reshape((1, *image.shape[:2])))
              for image in images]


# prepare the image for the VGG model
# convert the image pixels to a numpy array
image = img_to_array(image)
print(model.summary())

In [ ]:
# sorted(os.listdir(os.path.dirname(fmriprep_dir)+'/sub-3002498/'))
# Path(cimaq_files.bolds[0]).parts[-4:-2]
tpath=os.path.join(cimaq_files.bolds[0].split('fmriprep')[0],
                                     *Path(cimaq_files.bolds[0]).parts[-4:-2])
os.listdir(tpath)
events_path = glob.glob(os.path.join(events_dir,
                                     *Path(cimaq_files.bolds[0]).parts[-4:-2],
                            '*events.tsv'))[0]
events_path

Step 1: Load confound parameters outputed from the NIAK preprocessing pipeline (slow signal drift, motion parameters, mean white matter and mean ventricle signal intensity)

**Note: The full NIAK preprocessing pipeline scrubs (regresses out) motion outlier frames, which may not be compatible with other software like Nilearn or Nistats.
Intermediate data from the preprocessing pipeline should be used that have been slice-timed, co-registered and resampled (motion-corrected).  

These data are found under the **resample** directory (not under fMRI), in .nii format, with accompanying _extra.mat and confounds.tsv.gz files. 
These data have undergone **no smoothing**, and confounds have NOT been regressed out. 

Use gunzip *gz command to unzip .tsv files inside working directory

**Update: the nistats first-level model can model some of the slow drift and noise parameters (not used here). Here, I use all the confounds in *counfounds.tsv (including slow drift) as regressors, so I don't model slow drift, etc in the first-level model (it's redundant). 


In [ ]:
# fmriprep_dir = '/data/simexp/cimaq_preproc/fmriprep/'
# events_dir = '/data/simexp/CIMAQ_AS_BIDS/'
# atlases_dir = '../../nilearn_atlases/'

Step 2: create events variable, and events.tsv file from the 'sub-*_ses-4_task-memory_events.tsv' task file outputed by the cimaq_convert_eprime_to_bids_event.py script 

Number of rows = number of trials (first-level model uses onset times to match trial conditions to fMRI frames)

Documentation:
https://nistats.github.io/auto_examples/04_low_level_functions/write_events_file.html#sphx-glr-auto-examples-04-low-level-functions-write-events-file-py

Each encoding trial is modelled as a different condition (under trial_type column) so that it is modelled separately when creating the design matrix. The trial has its own column in the design matrix; the other columns = other trials (modelled together as single regressor), and confound regressors.

**Note: Some scans were cut short, meaning that the last few trials do not have associated brain activation frames, and they need to be left out of the analysis; 310 frames = full scan, 288 frames = incomplete (~15 participants).
"unscanned" trials need to be excluded from the model (about ~2-4 trials missing).
288*2.5 = 720s. 
Trial #115 (out of 117) offset time ~ 710s
Trial #116 (out of 117) onset ~ 723s

### Create unique trial identifiers & clean fMRI BOLD signal

In [ ]:

reduced_sig = pd.DataFrame([[np.mean(chunk) for chunk in chunks(row[1].values, 27)]
                            for row in signals.iterrows()], dtype=np.float16)


In [ ]:
ctl_idx = events00.iloc[:-1,:].loc[events00.condition=='Ctl'].index.tolist()
enc_idx = events00.iloc[:-1,:].loc[events00.condition=='Enc'].index.tolist()
X_ctl = apply_mask(list(nimage.iter_img(nimage.index_img(betamap00, ctl_idx))), vVS_mask)
X_enc = apply_mask(list(nimage.iter_img(nimage.index_img(betamap00, enc_idx))), vVS_mask)
# betamap00

In [ ]:
from nilearn import datasets
difumo256 = datasets.fetch_atlas_difumo(256, 3, data_dir=atlases_dir)
difumo_sorted = pd.DataFrame(difumo256.labels).iloc[:, 1:]
type(difumo256)
# yeo7_vis=difumo_sorted.loc[difumo_sorted.yeo_networks7=='VisCent'].difumo_names.index.tolist()

# # Lateral-Occipital Complex (LOC) - Not yet
# # Occipital Visual only
# vis_patt = re.compile('\.*occipital\.*|\.*visual\.*', re.I)
# vis_components = difumo_sorted.difumo_names.str.contains(vis_patt)
# vis_areas = vis_components.replace(False, pd.NA).dropna(how='all').index.tolist()
# vVS_mask = format_difumo([84],target_img=mask00,data_dir=atlases_dir)

In [ ]:
# from nilearn.input_data import NiftiMapsMasker
# # help(NiftiMapsMasker)
# maps_masker = NiftiMapsMasker(maps_img=nimage.index_img(nimage.load_img(difumo256.maps), [84]),
#                               standardize=False)
# time_series = maps_masker.fit_transform(mem_betamap, confounds=None)

In [ ]:
# [:-1] to remove ``constant`` 
# glm_betamap = nimage.concat_imgs(tuple(glm_model.compute_contrast(cont, output_type='effect_size')
#                                        for cont in tqdm(glm_model.design_matrices_[0].columns[:-1].tolist(),
#                                                         desc='computing contrasts')))
betamap00 = nimage.concat_imgs(tuple(model00.compute_contrast(cont, output_type='effect_size')
                                     for cont in tqdm(design00.columns[:-1].tolist(),
                                                    desc='computing contrasts')))

In [ ]:
memory_trials = events00.loc[events00.recognition_performance.notna()]
mem_indx = memory_trials.index.tolist()
mem_acc_labels = memory_trials.recognition_performance.values.tolist()[:-1]
mem_betamap = nimage.index_img(betamap, mem_indx[:-1])
mem_betamap.shape, len(mem_acc_labels)
enc_ctl_labels = [re.match('\w{3}',trial).group() for trial
                  in events00.condition.tolist()]
# events00.shape, betamap_str.shape
set(mem_acc_labels)

In [ ]:
# boolmask=nimage.new_img_like(ref_niimg=mask00,
#                                     data=v_roi_data)
visual_masker = nilearn.input_data.NiftiMasker(mask_img=visual_mask,
                                               mask_strategy='whole-brain-template')

In [ ]:
# enc_ctl = dict(tuple((cond,events00.condition.eq(cond).astype(int).values.tolist()+[0])
#                      for cond in events00.condition.unique()))
# len(enc_ctl['Ctl'])
# enc_ctl_betamap = model00.compute_contrast(enc_ctl['Enc'], output_type='effect_size')

In [ ]:
# mem_betamap.shape, maps_masker.fit_transform(mem_betamap).shape, len(mem_acc_labels)
nilearn.image.concat_imgs(mbbt55)

In [ ]:
classify_bmap(nilearn.image.concat_imgs(mbbt55), mask55, labels_list=idxs.values)

In [ ]:
# sub_svc = sklearn.svm.NuSVC() # 0.68 avg acc up to now
# sub_svc = sklearn.linear_model.SGDClassifier()
# 
# Cross-validation within 10 folds of training set
# predict
# class_weight='balanced'
cv_y_pred = cross_val_predict(sub_svc, X_train, y_train,
                              groups=y_train, cv=10)
# scores
cv_acc = cross_val_score(sub_svc, X_train, y_train,
                         groups=y_train, cv=10)
print(f'Cross-validation accuracy: {cv_acc}')

# evaluate overall model performance on training data
overall_acc = accuracy_score(y_pred = cv_y_pred, y_true = y_train)
overall_cr = classification_report(y_pred = cv_y_pred, y_true = y_train)
print(f'Accuracy\n{round(overall_acc, 3)}\n\nOverall Score\n{overall_cr}')

In [ ]:
from io import StringIO
pd.read_fwf(StringIO('\n'.join(list(filter(None,overall_cr.splitlines()[1:])))),
            sep='\\s+', header=None).set_axis(['Result','precision','recall',
                                               'f1-score','support'],
                                              axis=1).set_index('Result')

In [ ]:
# trial_betas = list(nimage.iter_img(betamap))[:-1]
# decoder = nilearn.decoding.FREMClassifier(standardize=False,
#                                           mask=mask00,
#                                           target_affine=cleaned_bold.affine,
#                                           target_shape=cleaned_bold.shape[:-1],
#                                           clustering_percentile = 20,
#                                           screening_percentile = 100,
#                                           mask_strategy='whole-brain-template')
# X_train, X_test, y_train, y_test = train_test_split(
#     trial_betas,
# #     apply_mask(trial_betas, mask00), # x
#     events00.condition, # y
#     test_size = 0.4, # 60%/40% split
#     shuffle = True, # shuffle dataset before splitting
#     stratify = events00.condition, # keep distribution of conditions consistent betw. train & test sets
#     #random_state = 123  # if set number, same shuffle each time, otherwise randomization algo
#     ) 
# decoder.fit(X_train, y_train)


In [ ]:
y_pred = decoder.predict(X_test)
# help(decoder.score)
# acc = decoder.score(X_test, y_test)
decoder_cr = classification_report(y_pred=y_pred, y_true=y_test) # get prec., recall & f1
# print results
print(decoder_cr)
# print(f'Accuracy\n{acc}\n{cr}')

In [ ]:
# model00.generate_report(contrasts=cdef,
#                         title=None, bg_img='MNI152TEMPLATE',
#                         threshold=3.09, alpha=0.001,
#                         cluster_threshold=0, height_control='fpr',
#                         min_distance=8.0, plot_type='slice', display_mode='ortho',
#                         report_dims=(1600, 800))

In [ ]:
# cont_def = dict(tuple((cond,events00.condition.str.contains(cond).astype(int).values)
#                       for cond in events00.condition.unique().tolist()))
# cont_def['Enc-Ctl'] = cont_def['Enc']-cont_def['Ctl']
# cont_def

In [ ]:
from nilearn.decoding import Decoder
# Here screening_percentile is set to 5 percent
# mask_img = haxby_dataset.mask
trial_betamap = nimage.concat_imgs(tuple(nimage.iter_img(betamap))[:-1])
decoder = Decoder(estimator='svc_l1', mask=mask00, smoothing_fwhm=None,
                  clustering_percentile = 20,
                  standardize=False, screening_percentile=5, scoring='accuracy')
decoder.fit(trial_betamap,
            events00.condition)
y_pred = decoder.predict(trial_betamap)

# Print the CV scores
print(decoder.cv_scores_['Enc'], decoder.cv_scores_['Ctl'])

In [ ]:
weight_img = decoder.coef_img_['Enc']
view_img(weight_img,
         title="SVM weights")
#          dim=-1)

In [ ]:

    


    # get map of coefficients    
    # coef_ = sub_svc.coef_
    # print(coef_.shape)
    #Return voxel weights into a nifti image using the NiftiMasker
    # coef_img = sub_masker.inverse_transform(coef_)
    #Save .nii to file
    # coef_img.to_filename(os.path.join(output_dir, 'Coef_maps', 'SVC_coeff_enc_ctl_sub-'+str(sub)+'.nii'))

enc_ctl_data = enc_ctl_data.append(pd.Series(s_data, index=enc_ctl_data.columns), ignore_index=True)

demo_data = sub_data.copy()
demo_data.reset_index(level=None, drop=False, inplace=True)

enc_ctl_data.insert(loc = 1, column = 'cognitive_status', value = demo_data['cognitive_status'],
                    allow_duplicates=True)
enc_ctl_data.insert(loc = 2, column = 'total_scrubbed_frames', value = demo_data['total_scrubbed_frames'],
                    allow_duplicates=True)
enc_ctl_data.insert(loc = 3, column = 'mean_FD', value = demo_data['mean_FD'], allow_duplicates=True)
enc_ctl_data.insert(loc = 4, column = 'hits', value = demo_data['hits'], allow_duplicates=True)
enc_ctl_data.insert(loc = 5, column = 'miss', value = demo_data['miss'], allow_duplicates=True)
enc_ctl_data.insert(loc = 6, column = 'correct_source', value = demo_data['correct_source'],
                    allow_duplicates=True)
enc_ctl_data.insert(loc = 7, column = 'wrong_source', value = demo_data['wrong_source'],
                    allow_duplicates=True)
enc_ctl_data.insert(loc = 8, column = 'dprime', value = demo_data['dprime'], allow_duplicates=True)
enc_ctl_data.insert(loc = 9, column = 'associative_memScore', value = demo_data['associative_memScore'],
                    allow_duplicates=True)    
    
enc_ctl_data.to_csv(os.path.join(output_dir, 'SVC_withinSub_enc_ctl_wholeBrain.tsv'),
    sep='\t', header=True, index=False)


In [ ]:
s_task = pd.read_csv(subject_data.events, sep='\t')
#rename "trial_type" column as "condition"
s_task.rename(columns={'trial_type':'condition'}, inplace=True)
confounds = Minimal().load(subject_data.bolds)
scanDur = confounds.shape[0]*2.5
#add columns to DataFrame
numCol = s_task.shape[1] 
insertCol = [4, numCol+1, numCol+2, numCol+3]
colNames = ['trial_type', 'unscanned', 'ctl_miss_hit', 'ctl_miss_ws_cs']
colVals = ['TBD', 0, 'TBD', 'TBD']
for i in range(0, len(colNames)):
    s_task.insert(loc=insertCol[i], column=colNames[i], value=colVals[i], allow_duplicates=True)

#The 'unscanned' column flag trials for which no brain data was acquired.
#The scan's duration is shorter than the trial's offset time
#(0 = data, 1 = no data)
for j in s_task[s_task['offset']>scanDur].index:
    s_task.loc[j, 'unscanned']=1

#pas trial numbers with zeros (on the left) to preserve trial 
#temporal order when trials are alphabetized
s_task['trial_number'] = s_task['trial_number'].astype('object',
                                                       copy=False)
for k in s_task.index:
    s_task.loc[k, 'trial_number'] = str(s_task.loc[k, 'trial_number']).zfill(3)

#trial_type should have a unique entry per row so that each trial 
#can be modelled as a separate condition
countEnc = 0
countCTL = 0
for m in s_task[s_task['condition']=='Enc'].index:
    countEnc = countEnc + 1
    s_task.loc[m, 'trial_type'] = 'Enc'+str(countEnc)
    if s_task.loc[m, 'position_accuracy'] == 0:
        s_task.loc[m, 'ctl_miss_hit']='missed'
        s_task.loc[m, 'ctl_miss_ws_cs']='missed'
    elif s_task.loc[m, 'position_accuracy'] == 1:
        s_task.loc[m, 'ctl_miss_hit']='hit'
        s_task.loc[m, 'ctl_miss_ws_cs']='wrongsource'
    elif s_task.loc[m, 'position_accuracy'] == 2:
        s_task.loc[m, 'ctl_miss_hit']='hit'
        s_task.loc[m, 'ctl_miss_ws_cs']='correctsource' 
for n in s_task[s_task['condition']=='CTL'].index:
    countCTL = countCTL + 1
    s_task.loc[n, 'trial_type'] = 'CTL'+str(countCTL)
    s_task.loc[n, 'ctl_miss_hit']='control'
    s_task.loc[n, 'ctl_miss_ws_cs']='control'

#78 encoding and 39 control trials if full scan
print('Number of encoding trials:  ', countEnc)
print('Number of control trials:  ', countCTL)
    
#keep only trials for which fMRI data was collected
s_task = s_task[s_task['unscanned']==0]

#Save vectors of trial labels (e.g., encoding vs control)
#to label trials for classification analyses
ttypes1 = s_task['condition']
# ttypes1.to_csv(outTask_dir+'/sub-'+id+'_enco_ctl.tsv',
#               sep='\t', header=True, index=False)

ttypes2 = s_task['ctl_miss_hit']
# ttypes2.to_csv(outTask_dir+'/sub-'+id+'_ctl_miss_hit.tsv',
#               sep='\t', header=True, index=False)

ttypes3 = s_task['ctl_miss_ws_cs']
# ttypes3.to_csv(outTask_dir+'/sub-'+id+'_ctl_miss_ws_cs.tsv',
#               sep='\t', header=True, index=False)


#from s_task dataframe, create an events dataframe to create a design matrix 
#that will be inputed into a first-level model in nistats
ev_cols = ['onset', 'duration', 'trial_type', 'condition', 'ctl_miss_hit', 
           'ctl_miss_ws_cs', 'trial_number']
all_events = s_task[ev_cols]  

print(all_events.shape)
print(all_events.iloc[0:30, :])


In [ ]:
conditions = dict(tuple((col, np.array(list(map(int,col==design_matrix0.columns))))
   for col in ['Enc','Ctl', 'constant']))
enc_minus_ctl = conditions['Enc'] - conditions['Ctl']
enc_minus_const = conditions['Enc'] - conditions['constant']
ctl_minus_const =  conditions['Ctl'] - conditions['constant']

**Step3 : Implement first-level model (implements regression in nistats), generate contrasts and output maps of beta values (parameter estimators; one map of betas per trial).

**About first-level model:

Note 1: nistats.first_level_model provides an interface to use the glm implemented in nistats.regression

Note 2: each encoding trial is modelled as a separate condition to obtain separate maps of beta values (model's output type to get betas = **effect sizes** (Nistats name), aka parameter estimates)

**UPDATE: a separate model is created to obtain a beta map for each trial. 
The trial of interest is modelled as a unique condition (it has its own column in the design matrix), and the other trials are modelled together.
Version A: beside the trial of interest (1 regressor), the control trials and encoding trials are modelled separately (2 regressors)
Version B: beside the trial of interest (1 regressor), the control trials and encoding trials are modelled together as a single "other_trials" condition (1 regressor)

Note 3: the first_level_model can either be given a pre-constructed **design matrix** (built from the events and confounds files in a separate preparatory step) as a parameter, or be given the **events and confounds** files directly as parameters (skipping the need to create a design matrix in a separate step). In the second situation, the model will generate and output the design matrix.  

As a parameter to the first-level-model, the design matrix takes precedence over the events and confounds parameters.

Nistats link on design matrices:
https://nistats.github.io/auto_examples/04_low_level_functions/plot_design_matrix.html

https://nistats.github.io/modules/generated/nistats.design_matrix.make_first_level_design_matrix.html

Example where the design matrix is created directly within the first level model interface by inputing events and confounds files directly as parameters (I'm not doing that here): https://nistats.github.io/auto_examples/01_tutorials/plot_first_level_model_details.html#running-a-basic-model

CHOSEN: 2-step method. Create design matrix directly from events file and confounds outputed from NIAK (resample directory), then generate and fit first-level-model

**About contrasts and maps of beta values (parameter estimators):

When each trial is modeled as a separate condition, it is represented as its own column in the design matrix. 

To access the estimated coefficients (betas of the GLM model), we need to specify "canonical contrasts" (one per trial) that isolate design matrix columns (each contrast has a single 1 in its corresponding colum, and 0s for all the other columns).

Sources of info: how to extract beta maps in nistats (instructions are a bit obscure)
https://15-35545854-gh.circle-artifacts.com/0/home/ubuntu/nistats/doc/_build/html/auto_examples/plot_spm_auditory.html
https://github.com/poldracklab/fitlins/pull/48
https://nistats.github.io/modules/generated/nistats.first_level_model.FirstLevelModel.html#nistats.first_level_model.FirstLevelModel


**Version A: A separate model created for each trial.
Trial of interest modelled as its own condition, and other trials are modelled as two conditions: control, and encoding (excluding trial of interest)


In [ ]:
#Apply 8mm smoothing since using unsmoothed preprocessed data 
#taken from NIAK's "resample" output directory
fmri_imgNS = load_img(subject_data.bolds)
fmri_img = image.smooth_img(fmri_imgNS, 8)

#Define parameters related to image acquisition
# time in seconds & number of frames
# Frame times for each epi scan
# alternatives to 'spm': 'glover', or 'spm + derivative'

# hrf_model, numTrials = 'spm', 

# This concatenation step might become obsolete, test Nilearn concatenation based on padded numbers!!
# Compile list of beta maps and their trial number, to be sorted by order of trial
all_betas_filelist_A = []

#Create a design matrix, first level model and beta map for each encoding and control trial 
# 117 trials if full scan (no missing frames)
for i in tqdm(list(range(all_events.shape[0]))):

    #copy all_events dataframe to keep the original intact
    events = all_events.copy(deep = True)
    #Determine trial number and condition (encoding or control)
    tnum, tname = events.iloc[i, 6], events.iloc[i, 2]
    #Version A: (2 conditions modelled separately)
    #modify trial_type column to model only the trial of interest
    events['trial_type'] = ['X_'+row[1].condition if row[1].trial_number == tnum
                            else row[1].condition for row in events.iterrows()]
    #verify: what determines the order of columns in design matrix?    
    #remove unecessary columns
    events = events[['onset', 'duration', 'trial_type']]
    #create design matrix
    design = make_first_level_design_matrix(**design_params)    
    #create & fit model - Should data be standardized?
    trial_model = FirstLevelModel(**glm_params).fit(fmri_img,
                                                    design_matrices=design)
    design_matrix = trial_model.design_matrices_[0]

    #sanity check: print design matrices and corresponding parameter labels
    #Contrast vector: 1 in design matrix column that corresponds to trial of interest, 0s elsewhere
    contrast_vec = np.repeat(0, design_matrix.shape[1])
    contrast_vec[0] = 1
    print(contrast_vec)
#     break
    #compute contrast's beta maps with ``model.compute_contrast``
    #https://nistats.github.io/modules/generated/nistats.first_level_model.FirstLevelModel.html
    b_map = trial_model.compute_contrast(contrast_vec,
                                         output_type='effect_size') #"effect_size" for betas
    all_betas_filelist_A.append(b_map)
#     b_name = os.path.join(outBeta_dir_A, 'betas_sub'+str(id)+'_Trial'+str(tnum)+'_'+tname+'.nii')
#     #export b_map .nii image in output directory
#     nibabel.save(b_map, b_name)
#     print(os.path.basename(b_name))
#     all_betas_filelist_A.append(b_name)

    
alltrials_betas_A = nibabel.funcs.concat_images(images=all_betas_filelist_A, check_affines=True, axis=None)
# print(alltrials_betas_A.shape)
# nibabel.save(alltrials_betas_A, os.path.join(outBeta_dir_A, 'concat_all_betas_sub'+str(id)+'.nii'))
    

In [ ]:
view_img(mean_img(alltrials_betas_A),bg_img=anat0, threshold='auto')

**Version B: A separate model created for each trial.
Trial of interest modelled as its own condition, and other trials (encoding and control, excluding trial of interest) are modelled as a single condition

In [ ]:
outBeta_dir_B = '/Users/mombot/Documents/Simexp/CIMAQ/Data/Nistats/Betas/122922/OneModelPerTrial_B'
all_betas_filelist_B = []
####

#Create a design matrix, first level model and beta map for each encoding and control trial 
for i in range (0, numTrials):

    #copy all_events dataframe to keep the original intact
    events = all_events.copy(deep = True)

    #Determine trial number and condition (encoding or control)
    tnum = events.iloc[i, 6]
    currentCondi = events.iloc[i, 3]
    tname = events.iloc[i, 2]
        
    #Version A: (2 conditions modelled separately)
    #modify trial_type column to model only the trial of interest 
    for j in events.index:
        if events.loc[j, 'trial_number'] != tnum:
            events.loc[j, 'trial_type']= 'X_otherCondi'
            #X for condition to remain in alphabetical order: trial of interest, X_CTL, X_Enc
    #verify: what determines the order of columns in design matrix?    

    #remove unecessary columns    
    cols = ['onset', 'duration', 'trial_type']
    events = events[cols]
    
    #create the model
    s_model = FirstLevelModel(t_r=tr, drift_model = None, standardize = True, noise_model='ar1',
                               hrf_model = hrf_model)    
    #Should data be standardized?

    #create the design matrices
    design = make_first_level_design_matrix(frame_times,
                                            events=events,
                                            drift_model=None,
#                                             add_regs=confounds, 
                                            hrf_model=hrf_model)
    
    #fit model with design matrix
    s_model = s_model.fit(fmri_img, design_matrices = design)
    
    design_matrix = s_model.design_matrices_[0]
    
    #sanity check: print design matrices and corresponding parameter labels
    #plot outputed design matrix for visualization
    print(str(tnum), ' ', tname, ' ', design_matrix.columns[0:3])
    plot_design_matrix(design_matrix)
    plt.show()

    #Contrast vector: 1 in design matrix column that corresponds to trial of interest, 0s elsewhere
    contrast_vec = np.repeat(0, design_matrix.shape[1])
    contrast_vec[0] = 1

    #compute the contrast's beta maps with the model.compute_contrast() method,
    #based on contrast provided. 
    #https://nistats.github.io/modules/generated/nistats.first_level_model.FirstLevelModel.html
    b_map = s_model.compute_contrast(contrast_vec, output_type='effect_size') #"effect_size" for betas
    b_name = os.path.join(outBeta_dir_B, 'betas_sub'+str(id)+'_Trial'+str(tnum)+'_'+tname+'.nii')
    #export b_map .nii image in output directory
    nibabel.save(b_map, b_name)
    print(os.path.basename(b_name))
    all_betas_filelist_B.append(b_name)
    
alltrials_betas_B = nibabel.funcs.concat_images(images=all_betas_filelist_B, check_affines=True, axis=None)
print(alltrials_betas_B.shape)
nibabel.save(alltrials_betas_B, os.path.join(outBeta_dir_B, 'concat_all_betas_sub'+str(id)+'.nii'))



In [ ]:
outBeta_dir = '/Users/mombot/Documents/Simexp/CIMAQ/Data/Nistats/Betas/122922/Conditions_Contrasts'

#Model 1: encoding vs control conditions
events1 = all_events.copy(deep = True)
cols = ['onset', 'duration', 'condition']
events1 = events1[cols]
events1.rename(columns={'condition':'trial_type'}, inplace=True)

print(events1.head())

#create the model
model1 = FirstLevelModel(t_r=tr, drift_model = None, standardize = True, noise_model='ar1',
                         hrf_model = hrf_model)    
#Should data be standardized?

#create the design matrices
design1 = make_first_level_design_matrix(frame_times, events=events1,
                                        drift_model=None, add_regs=confounds, 
                                        hrf_model=hrf_model)

#fit model with design matrix
model1 = model1.fit(fmri_img, design_matrices = design1)    

design_matrix1 = model1.design_matrices_[0]    
plot_design_matrix(design_matrix1)
plt.show()
print(design_matrix1.columns[0:5])

#Condition order: control, encoding (alphabetical)

#contrast 1.1: control condition
ctl_vec = np.repeat(0, design_matrix1.shape[1])
ctl_vec[0] = 1
b11_map = model1.compute_contrast(ctl_vec, output_type='effect_size') #"effect_size" for betas
b11_name = os.path.join(outBeta_dir, 'betas_sub'+str(id)+'_ctl.nii')
nibabel.save(b11_map, b11_name)
print(os.path.basename(b11_name))
print(ctl_vec)

#contrast 1.2: encoding condition
enc_vec = np.repeat(0, design_matrix1.shape[1])
enc_vec[1] = 1
b12_map = model1.compute_contrast(enc_vec, output_type='effect_size') #"effect_size" for betas
b12_name = os.path.join(outBeta_dir, 'betas_sub'+str(id)+'_enc.nii')
nibabel.save(b12_map, b12_name)
print(os.path.basename(b12_name))
print(enc_vec)

#contrast 1.3: encoding minus control 
encMinCtl_vec = np.repeat(0, design_matrix1.shape[1])
encMinCtl_vec[1] = 1
encMinCtl_vec[0] = -1
b13_map = model1.compute_contrast(encMinCtl_vec, output_type='effect_size') #"effect_size" for betas
b13_name = os.path.join(outBeta_dir, 'betas_sub'+str(id)+'_enc_minus_ctl.nii')
nibabel.save(b13_map, b13_name)
print(os.path.basename(b13_name))
print(encMinCtl_vec)


In [ ]:
#Model 2: missed vs hit encoding trials
events2 = all_events.copy(deep = True)
cols2 = ['onset', 'duration', 'ctl_miss_hit']
events2 = events2[cols2]
events2.rename(columns={'ctl_miss_hit':'trial_type'}, inplace=True)

print(events2.iloc[0:15, :])

#create the model
model2 = FirstLevelModel(t_r=tr, drift_model = None, standardize = True, noise_model='ar1',
                         hrf_model = hrf_model)    
#Should data be standardized?

#create the design matrices
design2 = make_first_level_design_matrix(frame_times, events=events2,
                                        drift_model=None, add_regs=confounds, 
                                        hrf_model=hrf_model)

#fit model with design matrix
model2 = model2.fit(fmri_img, design_matrices = design2)    

design_matrix2 = model2.design_matrices_[0]    
plot_design_matrix(design_matrix2)
plt.show()
print(design_matrix2.columns[0:5])

##Condition order: control, hit, missed (alphabetical)

#contrast 2.1: miss 
miss_vec = np.repeat(0, design_matrix2.shape[1])
miss_vec[2] = 1
b21_map = model2.compute_contrast(miss_vec, output_type='effect_size') #"effect_size" for betas
b21_name = os.path.join(outBeta_dir, 'betas_sub'+str(id)+'_miss.nii')
nibabel.save(b21_map, b21_name)
print(os.path.basename(b21_name))
print(miss_vec)

#contrast 2.2: hit 
hit_vec = np.repeat(0, design_matrix2.shape[1])
hit_vec[1] = 1
b22_map = model2.compute_contrast(hit_vec, output_type='effect_size') #"effect_size" for betas
b22_name = os.path.join(outBeta_dir, 'betas_sub'+str(id)+'_hit.nii')
nibabel.save(b22_map, b22_name)
print(os.path.basename(b22_name))
print(hit_vec)

#contrast 2.3: hit minus miss
hit_min_miss_vec = np.repeat(0, design_matrix2.shape[1])
hit_min_miss_vec[1] = 1
hit_min_miss_vec[2] = -1
b23_map = model2.compute_contrast(hit_min_miss_vec, output_type='effect_size') #"effect_size" for betas
b23_name = os.path.join(outBeta_dir, 'betas_sub'+str(id)+'_hit_minus_miss.nii')
nibabel.save(b23_map, b23_name)
print(os.path.basename(b23_name))
print(hit_min_miss_vec)

#contrast 2.4: hit minus control
hit_min_ctl_vec = np.repeat(0, design_matrix2.shape[1])
hit_min_ctl_vec[1] = 1
hit_min_ctl_vec[0] = -1
b24_map = model2.compute_contrast(hit_min_ctl_vec, output_type='effect_size') #"effect_size" for betas
b24_name = os.path.join(outBeta_dir, 'betas_sub'+str(id)+'_hit_minus_ctl.nii')
nibabel.save(b24_map, b24_name)
print(os.path.basename(b24_name))
print(hit_min_ctl_vec)

#contrast 2.5: miss minus control 
miss_min_ctl_vec = np.repeat(0, design_matrix2.shape[1])
miss_min_ctl_vec[2] = 1
miss_min_ctl_vec[0] = -1
b25_map = model2.compute_contrast(miss_min_ctl_vec, output_type='effect_size') #"effect_size" for betas
b25_name = os.path.join(outBeta_dir, 'betas_sub'+str(id)+'_miss_minus_ctl.nii')
nibabel.save(b25_map, b25_name)
print(os.path.basename(b25_name))
print(miss_min_ctl_vec)


In [ ]:
#Model 3: correct source vs wrong source encoding trials
events3 = all_events.copy(deep = True)
cols3 = ['onset', 'duration', 'ctl_miss_ws_cs']
events3 = events3[cols3]
events3.rename(columns={'ctl_miss_ws_cs':'trial_type'}, inplace=True)

print(events3.iloc[0:15, :])

#create the model
model3 = FirstLevelModel(t_r=tr, drift_model = None, standardize = True, noise_model='ar1',
                         hrf_model = hrf_model)    
#Should data be standardized?

#create the design matrices
design3 = make_first_level_design_matrix(frame_times, events=events3,
                                        drift_model=None, add_regs=confounds, 
                                        hrf_model=hrf_model)

#fit model with design matrix
model3 = model3.fit(fmri_img, design_matrices = design3)    

design_matrix3 = model3.design_matrices_[0]    
plot_design_matrix(design_matrix3)
plt.show()
print(design_matrix3.columns[0:5])

##Condition order: control, correct source, missed, wrong source (alphabetical)

#contrast 3.1: wrong source 
ws_vec = np.repeat(0, design_matrix3.shape[1])
ws_vec[3] = 1
b31_map = model3.compute_contrast(ws_vec, output_type='effect_size') #"effect_size" for betas
b31_name = os.path.join(outBeta_dir, 'betas_sub'+str(id)+'_ws.nii')
nibabel.save(b31_map, b31_name)
print(os.path.basename(b31_name))
print(ws_vec)

#contrast 3.2: correct source
cs_vec = np.repeat(0, design_matrix3.shape[1])
cs_vec[1] = 1
b32_map = model3.compute_contrast(cs_vec, output_type='effect_size') #"effect_size" for betas
b32_name = os.path.join(outBeta_dir, 'betas_sub'+str(id)+'_cs.nii')
nibabel.save(b32_map, b32_name)
print(os.path.basename(b32_name))
print(cs_vec)

#contrast 3.3: correct source minus wrong source
cs_minus_ws_vec = np.repeat(0, design_matrix3.shape[1])
cs_minus_ws_vec[1] = 1
cs_minus_ws_vec[3] = -1
b33_map = model3.compute_contrast(cs_minus_ws_vec, output_type='effect_size') #"effect_size" for betas
b33_name = os.path.join(outBeta_dir, 'betas_sub'+str(id)+'_cs_minus_ws.nii')
nibabel.save(b33_map, b33_name)
print(os.path.basename(b33_name))
print(cs_minus_ws_vec)

#contrast 3.4: correct source minus miss
cs_minus_miss_vec = np.repeat(0, design_matrix3.shape[1])
cs_minus_miss_vec[1] = 1
cs_minus_miss_vec[2] = -1
b34_map = model3.compute_contrast(cs_minus_miss_vec, output_type='effect_size') #"effect_size" for betas
b34_name = os.path.join(outBeta_dir, 'betas_sub'+str(id)+'_cs_minus_miss.nii')
nibabel.save(b34_map, b34_name)
print(os.path.basename(b34_name))
print(cs_minus_miss_vec)

#contrast 3.5: wrong source minus miss
ws_minus_miss_vec = np.repeat(0, design_matrix3.shape[1])
ws_minus_miss_vec[3] = 1
ws_minus_miss_vec[2] = -1
b35_map = model3.compute_contrast(ws_minus_miss_vec, output_type='effect_size') #"effect_size" for betas
b35_name = os.path.join(outBeta_dir, 'betas_sub'+str(id)+'_ws_minus_miss.nii')
nibabel.save(b35_map, b35_name)
print(os.path.basename(b35_name))
print(ws_minus_miss_vec)

#contrast 3.6: correct source minus control
cs_minus_ctl_vec = np.repeat(0, design_matrix3.shape[1])
cs_minus_ctl_vec[1] = 1
cs_minus_ctl_vec[0] = -1
b36_map = model3.compute_contrast(cs_minus_ctl_vec, output_type='effect_size') #"effect_size" for betas
b36_name = os.path.join(outBeta_dir, 'betas_sub'+str(id)+'_cs_minus_ctl.nii')
nibabel.save(b36_map, b36_name)
print(os.path.basename(b36_name))
print(cs_minus_ctl_vec)

#contrast 3.7: wrong source minus control
ws_minus_ctl_vec = np.repeat(0, design_matrix3.shape[1])
ws_minus_ctl_vec[3] = 1
ws_minus_ctl_vec[0] = -1
b37_map = model3.compute_contrast(ws_minus_ctl_vec, output_type='effect_size') #"effect_size" for betas
b37_name = os.path.join(outBeta_dir, 'betas_sub'+str(id)+'_ws_minus_ctl.nii')
nibabel.save(b37_map, b37_name)
print(os.path.basename(b37_name))
print(ws_minus_ctl_vec)


In [ ]:
#Visualize beta maps

#plotting brain images in nilearn:
#http://nilearn.github.io/plotting/index.html

#define directory where subject's functional mask and anatomical scan reside
anat_dir = '/Users/mombot/Documents/Simexp/CIMAQ/Data/anat/122922'
#subject's anatomical scan
anat = nibabel.load(os.path.join(anat_dir, 'anat_sub122922_nuc_stereonl.nii'))
plot_anat(anat)

beta_list = glob.glob(os.path.join(outBeta_dir, '*.nii'))

for beta in beta_list:
    print(beta)
    plot_img(beta)
    plot_stat_map(stat_map_img=beta, bg_img=anat, cut_coords=(0, 0, 0), threshold=0.001, colorbar=True)
    show()
